### Project Overview

Millions of patients book appointments in hospitals and clinics every day, but a significant number of them do not show up.

These missed appointments, known as “no-shows,” lead to:

- Wasted doctor and staff time  
- Delayed care for other patients  
- Financial losses for healthcare providers  
- Inefficient use of hospital resources

### Why Predicting No-Shows Matters

By analyzing historical appointment data, hospitals can:

- Send reminders to patients who are at higher risk of missing appointments  
- Schedule appointments more effectively  
- Identify patterns related to age, health conditions, and scheduling behavior  
- Improve timely access to care

### Project Goal

This project uses real-world data from over 110,000 hospital appointments in Brazil to:

- Analyze key trends and factors behind patient no-shows  
- Build a machine learning model to predict whether a patient will miss their appointment  
- Develop visual dashboards in Power BI to support hospital decision-making

### Impact

By predicting and reducing no-shows, healthcare providers can:

- Increase operational efficiency  
- Reduce waiting times  
- Improve resource management  
- Enhance patient care and satisfaction


<div style="border:1px solid #1E90FF; padding: 15px; border-radius: 5px; background-color: #E6F0FA">

<h3 style="color:#1E90FF;">Table of Contents</h3>

1. <a href="#importing-libraries-and-loading-data">Importing Libraries and Loading Dataset</a>  
2. <a href="#dataset-description">Dataset Description</a>  
3. <a href="#data-cleaning">Data Cleaning</a>  
4. <a href="#data-cleaning">Descriptive Statistics</a>  
5. <a href="#exploratory-data-analysis">Exploratory Data Analysis (EDA)</a>  
6. <a href="#machine-learning-pipeline">Machine Learning Pipeline</a>  
7. <a href="#Tableau-dashboard">Tableau Dashboard</a>  
8. <a href="#conclusion">Conclusion & Recommendations</a>

</div>


<div style="border:0.5px solid #1E90FF; padding: 10px 15px; border-radius: 5px; background-color:rgb(58, 118, 228); width: fit-content; font-size: 15px;">
  <h4 style="margin: 5px 0;">Importing Libraries and Loading Dataset</h4>
</div>

In [ ]:
### Data Handling
import pandas as pd
import numpy as np

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns

### Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
### Load the dataset
df = pd.read_csv("C:/Users/DEEPANSHI/OneDrive/Desktop/INTERNSHIP ELEVATE LABS/project 1/KaggleV2-May-2016.csv")


<div style="border:0.5px solid #1E90FF; padding: 10px 15px; border-radius: 5px; background-color:rgb(58, 118, 228); width: fit-content; font-size: 15px;">
  <h4 style="margin: 5px 0;">Dataset Description</h4>
</div>

In [ ]:

from IPython.display import display, HTML

# Set horizontal scroll for wide DataFrames
display(HTML("<style>.output_result {overflow-x: auto;}</style>"))
# Renaming columns to fix spelling and formatting
df.rename(columns={
    'Hipertension': 'Hypertension',
    'Handcap': 'Handicap'
}, inplace=True)
display(df)


This dataset contains medical appointment records from Brazil, and it's used to analyze whether patients show up for their scheduled appointments. The dataset has 110,527 rows and 14 columns.

1. PatientId
A unique identifier for each patient. It helps identify if a patient booked multiple appointments. It’s stored as a long number (in scientific notation), so we may convert it for readability.

2. AppointmentID
Another unique identifier — this one is for the appointment itself. It’s not very useful for analysis or prediction but helps keep records unique.

3. Gender
Shows the patient's gender: either 'M' for male or 'F' for female. This can help us analyze if gender affects appointment attendance.

4. ScheduledDay
The date and time when the patient scheduled their appointment. It's in string format and needs to be converted to datetime for calculations like lead time (how many days between scheduling and appointment).

5. AppointmentDay
The actual date when the appointment was supposed to happen. This, combined with ScheduledDay, helps calculate delays or behavior patterns.

6. Age
Age of the patient in years. It’s an important feature — for example, younger or elderly patients might have different attendance patterns. We'll need to check and clean any negative values here.

7. Neighbourhood
Shows where the clinic is located. This is the patient’s local area or zone. We can use this for location-based trends.

8. Scholarship
This is a binary column: 1 means the patient is part of a social welfare program, and 0 means they are not. It helps analyze if socio-economic factors influence no-shows.

9. Hypertension
 It’s 1 if the patient has high blood pressure, 0 if not. It's one of the medical condition features.

10. Diabetes
1 if the patient has diabetes, 0 if not. Another health-related factor that might affect attendance.

11. Alcoholism
1 if the patient has been flagged for alcoholism, 0 if not. May be related to reliability in attending medical appointments.

12. Handicap
This represents if a patient is physically disabled. The values can range from 0 to 4 (the higher the number, the more severe the disability). Again, it might affect ability or likelihood to attend appointments.

13. SMS_received
This tells whether the patient received a reminder SMS or not. 1 means SMS was received, 0 means it wasn’t. This is one of the most important predictors of a no-show.

14. No-show
This is the target variable — it tells whether the patient missed the appointment.

"No" means they did show up

"Yes" means they did not show up

<div style="border:0.5px solid #1E90FF; padding: 10px 15px; border-radius: 5px; background-color:rgb(58, 118, 228); width: fit-content; font-size: 15px;">
  <h4 style="margin: 5px 0;">Data Cleaning</h4>
</div>

In [ ]:
df.info()

In [ ]:
### Convert ScheduledDay and AppointmentDay to datetime format
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

# Create WaitingDays column
df['WaitingDays'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days

# Remove rows with negative ages
df = df[df['Age'] >= 0]

# Drop duplicates
df.drop_duplicates(inplace=True)



In [ ]:

df[['ScheduledDay', 'AppointmentDay', 'WaitingDays']].head()

In [ ]:
### Filtering Invalid Waiting Days 
### The `WaitingDays` column was calculated as the difference between the appointment date and the scheduling date. Any negative values in this column indicate data entry errors — where the appointment appears to have occurred before it was scheduled, which is not possible.
###To ensure data integrity, all records with `WaitingDays < 0` were removed from the dataset.
df = df[df['WaitingDays'] >= 0]
df[['WaitingDays']].head()



In [ ]:
### Remaining Rows After Cleaning

df.shape[0]
print ("Total Rows", df.shape[0])

<div style="border:0.5px solid #1E90FF; padding: 10px 15px; border-radius: 5px; background-color:rgb(58, 118, 228); width: fit-content; font-size: 15px;">
  <h4 style="margin: 5px 0;">Descriptive Statistics</h4>
</div>

<h5> After cleaning the dataset, we generate summary statistics of all numerical columns to understand central tendencies and data distribution. This helps identify any remaining outliers and guides further exploration.</h5>

In [ ]:
df.describe()


In [ ]:

def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    print(f"{column}: {len(outliers)} outliers")

# Run for only key columns
for col in ['Age', 'WaitingDays']:
    detect_outliers(df, col)


The cleaned dataset comprises 71,959 appointment records, including both patient-level and appointment-level variables. Key insights from the numerical features are as follows:

1.Patient Demographics

Age ranges from 0 to 115 years, with a mean of 38.5 and median of 39, indicating a wide age distribution centered around middle-aged individuals.

Presence of patients aged 0 and above 100 suggests the dataset includes both infants and elderly, which may warrant subgroup analysis in relation to appointment attendance.

2.Socioeconomic Indicators

The Scholarship variable has a mean of 0.09, indicating approximately 9% of patients are enrolled in a government assistance program.

This may serve as a proxy for socioeconomic status and could impact no-show behavior.

3.Medical Conditions

Hypertension is observed in 20.9% of patients, followed by Diabetes in 7.5%.

Alcoholism and Handicap are present in 2.5% and 2.0% of patients, respectively.

The Handicap variable ranges from 0 to 4, suggesting it may be categorical and could reflect varying degrees of disability.

4.Communication

The SMS_received feature has a mean of approximately 0.49, implying that around half of the patients received an SMS reminder.

This feature may be a key variable in assessing the effectiveness of reminder systems in reducing no-shows.

5.Waiting Time

WaitingDays range from 0 to 178 days, with:

Mean: 14.6 days

25th percentile: 3 days

Median: 8 days

75th percentile: 21 days

The variability in waiting time may influence appointment adherence and will be considered in the predictive modeling phase.



<div style="border:0.5px solid #1E90FF; padding: 10px 15px; border-radius: 5px; background-color:rgb(58, 118, 228); width: fit-content; font-size: 15px;">
  <h4 style="margin: 5px 0;">Exploratory Data Analysis (EDA)</h4>
</div>

Analyzing the target column No-show, which indicates whether a patient missed the appointment.

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='No-show', color='skyblue')
plt.title('Appointment Attendance Distribution')
plt.xlabel('No-show')
plt.ylabel('Count')
plt.show()


   <h4><B>BIVARIATE ANALYSIS</B></h4>

2. To check whether age influences patients miss appointments.

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='No-show', y='Age',hue='No-show', palette='pastel')
plt.title('Age Distribution by No-show Status')
plt.xlabel('No-show')
plt.ylabel('Age')
plt.show()


3.To check if sending an SMS helped people attend their appointments

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='SMS_received', hue='No-show', palette='Set2')
plt.title('No-show by SMS Received')
plt.xlabel('SMS Received (0 = No, 1 = Yes)')
plt.ylabel('Count')
plt.legend(title='No-show')
plt.show()


4. To check whether longer waiting time lead to more no-shows

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='No-show', y='WaitingDays',hue='No-show',palette='coolwarm',legend=False)
plt.title('Waiting Days by No-show Status')
plt.xlabel('No-show')
plt.ylabel('Waiting Days')
plt.show()


5. To check No-show Rates Among Scholarship Holders

In [ ]:

# Set plot style
plt.figure(figsize=(10,5))
sns.countplot(data=df, x='Scholarship', hue='No-show', palette='Set2')

# Add labels and title
plt.title('No-show by Scholarship Status')
plt.xlabel('Scholarship (0 = No, 1 = Yes)')
plt.ylabel('Count')
plt.legend(title='No-show')

plt.tight_layout()
plt.show()


In [ ]:
scholarship_group = df.groupby('Scholarship')['No-show'].value_counts(normalize=True).unstack() *100
print(scholarship_group)


6. Analyze the data to get the relation between Gender and No-show

In [ ]:
# Set plot style
plt.figure(figsize=(10,5))
sns.countplot(data=df, x='Gender', hue='No-show', palette='Set2')

# Add labels and title
plt.title('No-show by Gender')
plt.xlabel('Gender (Male, Female)')
plt.ylabel('Number of Appointments')
plt.legend(title='No-show')

plt.tight_layout()
plt.show()

In [ ]:
gender_group = df.groupby('Gender')['No-show'].value_counts(normalize=True).unstack()*100 
print(gender_group)


The dataset contains appointment records with a No-show column indicating whether a patient missed their scheduled appointment. After data cleaning, a total of 71,959 records remained.

Overall, 79.81% of patients showed up for their appointments, while 20.19% did not.

-- Age Analysis: The average patient age is approximately 38.5 years, with ages ranging from infants (0 years) to elderly patients (up to 115 years). Patients who missed their appointments tended to have a slightly lower median age compared to those who attended.

-- Waiting Days: The number of days between scheduling and the actual appointment ranges from 0 to 178 days. It was observed that patients with longer waiting periods were more likely to miss their appointments.

-- SMS Reminders: About 50% of the patients received SMS reminders. Those who received reminders had a slightly higher attendance rate, suggesting reminders may positively influence appointment adherence, although the effect was not very significant.

-- Scholarship Analysis: The Scholarship variable indicates whether a patient is enrolled in a government health support program.

Non-scholarship patients: 80.19% showed up, 19.81% missed.

Scholarship recipients: 76.26% showed up, 23.74% missed.

-- Gender Analysis: Gender did not show a significant impact on appointment attendance.

Female patients: 79.69% showed up, 20.31% missed.

Male patients: 80.03% showed up, 19.97% missed.


This analysis suggests that scholarship holders may be slightly more likely to miss appointments, which could reflect socio-economic or access-related factors worth further exploration.

<div style="border:0.5px solid #1E90FF; padding: 10px 15px; border-radius: 5px; background-color:rgb(58, 118, 228); width: fit-content; font-size: 15px;">
  <h4 style="margin: 5px 0;">Machine Learning Pipeline</h4>
</div>

Feature Engineering

In [ ]:
### Make a copy to preserve the original data
df_ml = df.copy()

### Encode target variable: 'No' → 0 (show), 'Yes' → 1 (no-show)
df_ml['No-show'] = df_ml['No-show'].map({'No': 0, 'Yes': 1})

### Convert 'Gender' into binary: 'F' → 0, 'M' → 1
df_ml['Gender'] = df_ml['Gender'].map({'F': 0, 'M': 1})

### Drop irrelevant features that don’t help in prediction
df_ml = df_ml.drop(['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay', 'Neighbourhood'], axis=1)

# View the cleaned features
df_ml.head()

In [ ]:
from sklearn.model_selection import train_test_split  ### to split data

### X = all columns except 'No-show'
### y = only 'No-show' column (this is what we want to predict)
X = df_ml.drop('No-show', axis=1)
y = df_ml['No-show']

### split into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# show shape of the datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Model Building with Decision Tree

In [ ]:
### Import the required libraries

### 1. Import the model
from sklearn.tree import DecisionTreeClassifier

### 2. Initialize the classifier
model = DecisionTreeClassifier(random_state=42)

### 3. Fit (train) the model on training data
model.fit(X_train, y_train)


MODEL EVALUATION

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Make predictions on the test set
y_pred = model.predict(X_test)

# 2. Check accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# 3. Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# 4. Classification Report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


In [ ]:
from IPython.display import Image, display
display(Image(filename='C:/Users/DEEPANSHI/OneDrive/Desktop/INTERNSHIP ELEVATE LABS/project 1/dashboard.jpg'))



8.Recommendations

- Increase SMS reminder coverage to more patients — current coverage is only ~49.3%.

- Consider sending multiple reminders to reduce no-shows, especially for younger age groups.

- Reduce no-show risk by offering appointments within 2–3 days of booking, especially for repeat patients.

- Add options for patients to easily cancel/reschedule appointments via app or SMS, which reduces unnotified no-shows.
